## Testing rdataframe tracktree reading in swan
First we define useful functions

In [ ]:
gROOT->ProcessLine(".x /eos/experiment/sndlhc/users/aiulian/fedraxswan/macros/rootlogon.C")

In [ ]:
EdbSegP GetFirstSegment(TClonesArray sf){
  //getting first segment, equivalent to t., which creates problem to RDataFrame due to branch name
  EdbSegP* firstseg = (EdbSegP*) sf.At(0);
  EdbSegP tr(*firstseg);
  return tr;
}

In [ ]:
TString path_prefix = TString(getenv("CERNBOX_HOME"));
path_prefix.Data()

In [ ]:
TFile *tracksfile = TFile::Open((path_prefix+TString("/SNDCosmics_tutorial/b000001/b000001.0.0.0.trk.root")).Data());
TTree *trackstree = (TTree*) tracksfile->Get("tracks");
ROOT::RDataFrame df(*trackstree);

In [ ]:
auto colNames = df.GetColumnNames();
// Print columns' names
for (auto &&colName : colNames) std::cout << colName << std::endl;

In [ ]:
auto dftr = df.Define("tr",GetFirstSegment,{"sf"});

## Definition of variables
Here, it is useful to define the variables we want to access. They can be transformed, when required (3D angle, change of units, etc.).

Calling the attributes directly (aka "tr.eTX") is not supported and not recommended. Better to use accessors ("tr.TX()")

In [ ]:
auto dftr1 = dftr.Define("TX","tr.TX()  ")
                 .Define("TY","tr.TY()  ")
                 .Define("Xcm","tr.X()*1e-4  ")
                 .Define("Ycm","tr.Y()*1e-4  ");

In [ ]:
//3D angles
auto dftr2 = dftr1.Define("tantheta","tr.Theta()").Define("theta","TMath::ATan(tantheta)");

//selecting good tracks (aka long, in this case
auto dfgoodtr = dftr2.Filter("nseg>2");

In [ ]:
auto htx = dfgoodtr.Histo1D({"htx","TX of tracks;TX", 75,1.5,1.5},"TX");
TCanvas *ctx = new TCanvas("ctx","TX angle");
htx->DrawClone();
ctx->Draw();

In [ ]:
auto hty = dfgoodtr.Histo1D({"hty","TY of tracks;TY", 75,1.5,1.5},"TY");
TCanvas *cty = new TCanvas("cty","TY angle");
hty->DrawClone();
cty->Draw();

In [ ]:
hty->GetEntries()

In [ ]:
auto hsz = dfgoodtr.Histo1D("s.eZ");
TCanvas *cz = new TCanvas("cz","Z Position of segments");
hsz->DrawClone();
cz->Draw();